In [1]:
#Add parent and grandparent folder to sys.path and get variable name
import os
import sys
import inspect

parentdir = os.path.abspath('..')
grandparentdir=os.path.dirname(parentdir)
sys.path.insert(0, parentdir)
sys.path.insert(0, grandparentdir)

var_name=os.path.basename(parentdir)

var_name

'consumpinit_scoe_gj_per_hh_residential_heat_energy'

In [2]:
import pandas as pd
import numpy as np

df_countries=pd.read_csv("../row_data/Countries_ISO3.csv")
df_countries

,Country Name,ISO3
0,Aruba,ABW
1,Africa Eastern and Southern,AFE
2,Afghanistan,AFG
3,Africa Western and Central,AFW
4,Angola,AGO
...,...,...
261,Kosovo,XKX
262,"Yemen, Rep.",YEM
263,South Africa,ZAF
264,Zambia,ZMB


In [3]:
df_WEBH=pd.read_excel("../row_data/WorldEnergyBalancesHighlights2021.xlsx",'TimeSeries_1971-2020')
df_WEBH.columns = df_WEBH.iloc[0]
#df_WEBH.replace(to_replace='..', value=0,inplace=True)
df_WEBH.head()

,Country,Product,Flow,NoCountry,NoProduct,NoFlow,1971,1972,1973,1974,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020 Provisional
0,Country,Product,Flow,NoCountry,NoProduct,NoFlow,1971,1972,1973,1974,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020 Provisional
1,Australia,"Coal, peat and oil shale",Production (PJ),01. Australia,"01. Coal, peat and oil shale",01. Production (PJ),1368.251838,1648.414248,1685.223676,1640.362098,...,9782.067241,10272.36221,11074.77871,11950.78794,12501.00154,12223.55369,12264.16248,12034.6322,12596.41582,12331.64135
2,Australia,"Coal, peat and oil shale",Imports (PJ),01. Australia,"01. Coal, peat and oil shale",02. Imports (PJ),0,0,0,0,...,6.755636,3.635428,2.57749,5.287409,6.978797,6.829994,7.916741,12.735731,17.037944,19.629498
3,Australia,"Coal, peat and oil shale",Exports (PJ),01. Australia,"01. Coal, peat and oil shale",03. Exports (PJ),-541.424001,-614.718001,-738.939398,-745.382946,...,-7720.391726,-8171.019358,-9101.059456,-10166.8869,-10634.4478,-10552.5051,-10248.0298,-10333.3662,-10629.0366,-10541.275
4,Australia,"Coal, peat and oil shale",Total energy supply (PJ),01. Australia,"01. Coal, peat and oil shale",04. Total energy supply (PJ),884.29747,923.250159,945.54188,928.32307,...,2013.814178,1968.848884,1831.204342,1736.61351,1797.035545,1863.051458,1872.030679,1797.650326,1744.105366,1648.774748


In [19]:
df_reselec = df_WEBH.loc[(df_WEBH['Product']=='Heat') & (df_WEBH['Flow']=='Residential (PJ)')] 
#df_reselec = df_reselec.drop(labels=df_reselec.loc[:, (df_reselec == '..').any()], axis=1)
df_reselec = df_reselec.replace('..', np.nan)
df_reselec = df_reselec.replace(0, np.nan)
# create a list of column names to keep
keep_cols = ['Country', 2015,2016,2017,2018,2019]

# find the intersection of the list of column names to keep and the list of all column names
cols_to_drop = list(set(df_reselec.columns) - set(keep_cols))

# drop the columns using the loc accessor
df_reselec = df_reselec.loc[:, keep_cols]

df_reselec.set_index('Country', inplace=True)

df_reselec = df_reselec.dropna(how='all')

df_reselec = df_reselec.reset_index()

df_reselec.head()

,Country,2015,2016,2017,2018,2019
0,Austria,31.476254,32.618672,34.549076,32.504213,32.310268
1,Belgium,0.336698,0.269802,0.285799,0.637202,0.683901
2,Czech Republic,42.545001,44.252998,44.627001,41.797591,40.663541
3,Denmark,67.829886,69.675239,70.063866,69.750409,67.709306
4,Estonia,11.869000,13.057001,13.319999,13.500002,13.680001


In [20]:
df_nhh = pd.read_excel("../row_data/Number of Households.xlsx")
df_nhh.columns = df_nhh.iloc[0]
df_nhh = df_nhh.drop(0)

df_nhh = df_nhh.rename(columns={'Yearly Data': 'Nation'})
df_nhh.set_index('Nation', inplace=True)
df_nhh = df_nhh.replace('...', np.nan)
df_nhh = df_nhh.replace('locked', np.nan)
df_nhh = df_nhh.replace('mil', np.nan)

df_nhh = df_nhh.dropna(how='all')
df_nhh = df_nhh.dropna(how='all',axis=1)
df_nhh = df_nhh.reset_index()

# create a list of names to keep
keep_country = df_reselec['Country'].values

# create a boolean mask
mask = df_nhh['Nation'].isin(keep_country)

# filter the DataFrame using the mask
df_nhh = df_nhh[mask]
df_nhh.head()

,Nation,2015,2016,2017,2018,2019
5,Austria,3.745341,3.768095,3.789855,3.810507,3.830037
11,Belgium,4.732743,4.754812,4.776881,4.798950,4.821018
31,Denmark,2.641332,2.652483,2.663635,2.674787,2.685939
35,Estonia,0.572219,0.569882,0.567545,0.565208,0.562871
37,Finland,2.627815,2.645751,2.663688,2.681625,2.699561


In [21]:
# create a list of names to keep
keep_country = df_nhh['Nation'].values

# create a boolean mask
mask = df_reselec['Country'].isin(keep_country)

# filter the DataFrame using the mask
df_reselec = df_reselec[mask]

df_reselec.head()

,Country,2015,2016,2017,2018,2019
0,Austria,31.476254,32.618672,34.549076,32.504213,32.310268
1,Belgium,0.336698,0.269802,0.285799,0.637202,0.683901
3,Denmark,67.829886,69.675239,70.063866,69.750409,67.709306
4,Estonia,11.869000,13.057001,13.319999,13.500002,13.680001
5,Finland,64.714999,70.539001,69.515999,67.413999,66.659998


In [22]:
years1=[c for c in list(df_reselec.columns) if type(c)==int]
years1
mean_WEBH={}
for y in years1:
    mean_WEBH[y]=np.mean(df_reselec[y])
mean_WEBH

{2015: 36.63735349780001,
 2016: 38.556330076554545,
 2017: 38.60085688045456,
 2018: 36.978970494745454,
 2019: 36.83152034309091}

In [23]:
years=[c for c in list(df_nhh.columns) if type(c)==int]
years
mean_nhh={}
for y in years:
    mean_nhh[y]=np.mean(df_nhh[y])
mean_nhh

{2015: 10.88366374034777,
 2016: 10.950905833731992,
 2017: 11.01790715076706,
 2018: 11.084672081199553,
 2019: 11.151401046324901}

In [24]:
#### HISTORICAL ######

countries=df_countries['Country Name'].values
ISO3=np.unique(df_countries['ISO3'].values)

coun_year=[]
for count in zip(countries,ISO3):
    for year in years:
        coun_year.append([count[0],count[1],year,0])
df_historical_out = pd.DataFrame(coun_year, columns=['Nation','iso_code3', 'Year',var_name])
df_historical_out

,Nation,iso_code3,Year,consumpinit_scoe_gj_per_hh_residential_heat_energy
0,Aruba,ABW,2015,0
1,Aruba,ABW,2016,0
2,Aruba,ABW,2017,0
3,Aruba,ABW,2018,0
4,Aruba,ABW,2019,0
...,...,...,...,...
1325,Zimbabwe,ZWE,2015,0
1326,Zimbabwe,ZWE,2016,0
1327,Zimbabwe,ZWE,2017,0
1328,Zimbabwe,ZWE,2018,0


In [25]:
for c,y in zip(df_historical_out['Nation'].values,df_historical_out['Year'].values):
    if c in df_reselec['Country'].values:
        reselec_gj= df_reselec.loc[(df_reselec['Country']==c)][y].values[0]*1e6
        hh_mil= df_nhh.loc[(df_nhh['Nation']==c)][y].values[0]*1e6
        dato=reselec_gj/hh_mil
        df_historical_out.loc[(df_historical_out['Nation'] == c) & 
                        (df_historical_out['Year']== y), var_name] = dato
        
    else:
        reselec_gj=mean_WEBH[y]*1e6
        hh_mil=mean_nhh[y]*1e6
        dato=reselec_gj/hh_mil
        df_historical_out.loc[(df_historical_out['Nation'] == c) & 
                        (df_historical_out['Year']== y), var_name] = dato
df_historical_out

,Nation,iso_code3,Year,consumpinit_scoe_gj_per_hh_residential_heat_energy
0,Aruba,ABW,2015,3.366270
1,Aruba,ABW,2016,3.520835
2,Aruba,ABW,2017,3.503465
3,Aruba,ABW,2018,3.336046
4,Aruba,ABW,2019,3.302860
...,...,...,...,...
1325,Zimbabwe,ZWE,2015,3.366270
1326,Zimbabwe,ZWE,2016,3.520835
1327,Zimbabwe,ZWE,2017,3.503465
1328,Zimbabwe,ZWE,2018,3.336046


In [26]:
df_historical_out.to_csv('../input_to_sisepuede/historical/'+var_name+'.csv')

In [27]:
#### Projected ######

countries=df_countries['Country Name'].values
ISO3=np.unique(df_countries['ISO3'].values)
years2=range(max(years)+1,2051)

coun_year=[]
for count in zip(countries,ISO3):
    for year in years2:
        coun_year.append([count[0],count[1],year])
df_projected_out = pd.DataFrame(coun_year, columns=['Nation','iso_code3', 'Year'])
df_projected_out[var_name]=0
df_projected_out

,Nation,iso_code3,Year,consumpinit_scoe_gj_per_hh_residential_heat_energy
0,Aruba,ABW,2020,0
1,Aruba,ABW,2021,0
2,Aruba,ABW,2022,0
3,Aruba,ABW,2023,0
4,Aruba,ABW,2024,0
...,...,...,...,...
8241,Zimbabwe,ZWE,2046,0
8242,Zimbabwe,ZWE,2047,0
8243,Zimbabwe,ZWE,2048,0
8244,Zimbabwe,ZWE,2049,0


In [28]:
for count,year in zip(df_projected_out['Nation'].values,df_projected_out['Year'].values):
    df_dato=df_historical_out[(df_historical_out['Nation']==count) & 
                               (df_historical_out['Year']==max(years))]
    dato=df_dato[var_name].values[0]
    df_projected_out.loc[(df_projected_out['Nation'] == count) & 
                        (df_projected_out['Year']== year), var_name] = dato
df_projected_out

,Nation,iso_code3,Year,consumpinit_scoe_gj_per_hh_residential_heat_energy
0,Aruba,ABW,2020,3.30286
1,Aruba,ABW,2021,3.30286
2,Aruba,ABW,2022,3.30286
3,Aruba,ABW,2023,3.30286
4,Aruba,ABW,2024,3.30286
...,...,...,...,...
8241,Zimbabwe,ZWE,2046,3.30286
8242,Zimbabwe,ZWE,2047,3.30286
8243,Zimbabwe,ZWE,2048,3.30286
8244,Zimbabwe,ZWE,2049,3.30286


In [29]:
df_projected_out.to_csv('../input_to_sisepuede/projected/'+var_name+'.csv')